In [1]:
import sys
sys.path.append('../app')
import cipher

from base64 import b64encode, b64decode
import bcrypt
import csv
import MySQLdb
import sshtunnel
from tqdm.notebook import tqdm

In [2]:
BIND_PORT = 4360

In [3]:
tunnel = sshtunnel.open_tunnel(
    os.environ.get('SSH_HOST'),
    ssh_username=os.environ.get('SSH_USER'),
    ssh_password=os.environ.get('SSH_PASSWORD'),
    local_bind_address=('127.0.0.1', BIND_PORT),
    remote_bind_address=(os.environ.get('MYSQL_DB_HOST'), 3306)
)

In [21]:
## originally used to add all users to DB

# fn = '../data/unicorn-keys-2.csv'

# with tunnel:
#     db = MySQLdb.connect(
#         user = os.environ.get('MYSQL_DB_USER'),
#         password = os.environ.get('MYSQL_DB_PASSWORD'),
#         database = os.environ.get('MYSQL_DB_DATABASE'),
#         host = '127.0.0.1',
#         port = BIND_PORT
#     )
#     cur = db.cursor()

#     nrows = 0
#     with open(fn) as f:
#         for _ in csv.reader(f):
#             nrows += 1
    
#     with open(fn) as f:   
#         for row in tqdm(csv.reader(f), total=nrows):
#             name = row[0]
#             fullname = row[1]
#             pw = fullname + '$' + row[2]
#             keyhash = bcrypt.hashpw(row[2].encode(), bcrypt.gensalt()).decode()
            
#             cur.execute(
#                 "insert into persons (name, fullname, secret_key_hash) values (%s, %s, %s)",
#                 (name, fullname, keyhash)
#             )

#     db.commit()
#     cur.close()
#     db.close()

In [42]:
### originally used to add RSA keys for all users to DB

fn = '../data/unicorn-keys.csv'

with tunnel:
    db = MySQLdb.connect(
        user = os.environ.get('MYSQL_DB_USER'),
        password = os.environ.get('MYSQL_DB_PASSWORD'),
        database = os.environ.get('MYSQL_DB_DATABASE'),
        host = '127.0.0.1',
        port = BIND_PORT
    )
    cur = db.cursor()

    nrows = 0
    with open(fn) as f:
        for _ in csv.reader(f):
            nrows += 1
    
    with open(fn) as f:
        for row in tqdm(csv.reader(f), total=nrows):
            pw = b64decode(row[2])[:18]
            pid = int.from_bytes(b64decode(row[2])[18:])
            priv, pub = cipher.generate_key_pair()
            public_key = cipher.serialize_public_key(pub)
            encrypted_private_key = cipher.encrypt_private_key(priv, pw)
            keyhash = bcrypt.hashpw(pw, bcrypt.gensalt()).decode()

            cur.execute(
                "update persons set secret_key_hash = %s where id = %s",
                (keyhash, pid)
            )
            cur.execute(
                "update persons set public_key = %s where id = %s",
                (public_key, pid)
            )
            cur.execute(
                "update persons set encrypted_private_key = %s where id = %s",
                (encrypted_private_key, pid)
            )

    db.commit()
    cur.close()
    db.close()

  0%|          | 0/63 [00:00<?, ?it/s]

In [5]:
# nuke and reset keys for the test user

with tunnel:
    db = MySQLdb.connect(
        user = os.environ.get('MYSQL_DB_USER'),
        password = os.environ.get('MYSQL_DB_PASSWORD'),
        database = os.environ.get('MYSQL_DB_DATABASE'),
        host = '127.0.0.1',
        port = BIND_PORT
    )
    cur = db.cursor()

    pw = b'antithermodynamics'
    pid = 63
    priv, pub = cipher.generate_key_pair()
    public_key = cipher.serialize_public_key(pub)
    encrypted_private_key = cipher.encrypt_private_key(priv, pw)
    keyhash = bcrypt.hashpw(pw, bcrypt.gensalt()).decode()

    cur.execute(
        "update persons set secret_key_hash = %s where id = %s",
        (keyhash, pid)
    )
    cur.execute(
        "update persons set public_key = %s where id = %s",
        (public_key, pid)
    )
    cur.execute(
        "update persons set encrypted_private_key = %s where id = %s",
        (encrypted_private_key, pid)
    )

    db.commit()
    cur.close()
    db.close()